# MathBERT

In [1]:
import os
import re

Index = str("4") # GPU Index/Indices

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= Index

import numpy as np
from scipy import stats
import torch
import torch.nn as nn 
from tqdm.notebook import tqdm
from transformers import *
import utils

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## Tokenization

In [4]:
### Configuration
# model_name_or_path = 'allenai/scibert_scivocab_cased'
model_name_or_path = 'bert-base-cased'
PrecedingSentNum = 7
SucceedingSentNum = 0
AddNotationToVocab = True

data_files = {}

data_files["train"] = "../../Data/features_s2orc_external_cs_limit=1000.json"

In [5]:
model = AutoModelForMaskedLM.from_pretrained(
    model_name_or_path,
    from_tf=bool(".ckpt" in model_name_or_path),
)
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
    use_fast=True,
)

### Add Custom Vocabs
tokenizer.add_tokens(["SYMBOL"])
tokenizer.add_tokens(["SECTION"])
tokenizer.add_tokens(["EQUATION"])
tokenizer.add_tokens(["CITATION"])
tokenizer.add_tokens(["$$"])

# SplitMethod can be "ByPaperID", 
PreprocessedData, added_tokens = utils.Preprocessing(tokenizer, data_files["train"], SplitMethod="ByPaperID",
                                       PrecedingSentNum=PrecedingSentNum, SucceedingSentNum=SucceedingSentNum,
                                       AddNotationToVocab=AddNotationToVocab)
tokenizer.add_tokens(added_tokens)
train_src, train_tgt, valid_src, valid_tgt, test_src, test_tgt = PreprocessedData

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



AddNotationsToVocab


  6%|▋         | 33/526 [00:00<00:01, 325.14it/s]

AddedTokens ['\\rightrightarrows', '\\subsetneqq', '\\deg', '\\blacksquare', '\\bowtie', '\\lim', '\\uplus', '\\eta', '\\textsf', '\\bigoplus', '\\rightarrow', '\\cr', '\\cap', '\\vphantom', '\\vee', '\\mid', '\\of', '\\textmd', '\\mathbfcal', '\\dots', '\\Psi', '\\mathbin', '\\right\\Vert', '\\ogreaterthan', '\\mathbf', '\\tt', '\\iint', '\\cup', '\\\\a', '\\\\v\\\\w', '\\iff', '\\\\\\qquad', '\\cite', '\\diagdown', '\\mu', '\\overrightarrow', '\\mbox', '\\makebox', '\\Bigm', '\\\\\\\\', '\\it', '\\scshape', '\\color', '\\leqslant', '\\index', '\\normalfont', '\\left\\Vert', '\\left\\lfloor', '\\div', '\\acute', '\\textstyle', '\\dashv', '\\ker', '\\Re', '\\small', '\\bigtriangleup', '\\spadesuit', '\\left\\bad', '\\dag', '\\\\\\nonumber', '\\bot', '\\complement', '\\Huge', '\\simeq', '\\ne', '\\phantom', '\\odot', '\\em', '\\left\\lbrace', '\\rightsquigarrow', '\\\\w', '\\footnotesize', '\\langle', '\\hline', '\\\\\\mathbf', '\\\\K', '\\textbf', '\\pazocal', '\\frac', '\\ddot', '\\\\

  0%|          | 52/222401 [00:00<07:13, 513.21it/s]


Prepare Data


100%|██████████| 222401/222401 [07:23<00:00, 501.75it/s] 


In [6]:
from datasets import Dataset, DatasetDict

MaxTokenLen = 512

def tokenize_function(examples):
    examples["text"] = tokenizer.convert_tokens_to_string(examples["text"].split())
    return tokenizer(
        examples["text"],
        truncation=False,
        max_length=MaxTokenLen,
        add_special_tokens = False,
        return_special_tokens_mask=False,
    )

## Test set Prediction

In [ ]:
### Configuration
model_name_or_path = 'bert-base-cased'

Verbose = True
MaskNumConstraint = 10
FineTune = True
Test = True
EpochNum = 20

In [ ]:
from termcolor import colored
from datasets.utils.logging import set_verbosity_error
import random
import copy

### Reload
model = AutoModelForMaskedLM.from_pretrained(
    model_name_or_path,
    from_tf=bool(".ckpt" in model_name_or_path),
)
model.resize_token_embeddings(len(tokenizer))
model = nn.DataParallel(model).to(device)

if FineTune:
    LossFunction = nn.CrossEntropyLoss()
    Optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.module.parameters()), lr=5e-6)
    pbar1 = tqdm(total=EpochNum)
    TopAcc = 0.
    for ep in range(EpochNum):
        ### Variables
        total_loss = []
        #####
        model.train()
        pbar2 = tqdm(total=len(train_tgt), leave=False)
        for i in range(len(train_tgt)):
            MaskNum = np.sum(np.array(train_src[i].split()) == "[MASK]")
            if MaskNum > MaskNumConstraint: # MASK Length Constraint
                pbar2.update(1); continue
            elif MaskNum == 0:
                pbar2.update(1); continue
            DataDict_train = { "train": Dataset.from_dict({ "text": [' '.join(train_src[i].split()) ] }), }
            datasets_train = DatasetDict(DataDict_train)

            tokenized_datasets = datasets_train.map(
                tokenize_function,
                batched=False,
                num_proc=1,
            )
            
            pred_token = train_src[i].split()
            tgt_token = train_tgt[i].split()
            
            for _ in range(1): # Partial Masking
                for cnt, x in enumerate(tokenized_datasets["train"]):
                    MASK_idx = [ii for ii, iids in enumerate(x["input_ids"])
                                if iids == tokenizer.convert_tokens_to_ids(["[MASK]"])[0]]
                    outputs = model(torch.tensor(x["input_ids"]).unsqueeze(0).to(device),)

                p = []
                loss = 0
                for m in MASK_idx:
                    loss += LossFunction(outputs["logits"][0][m:m+1],
                                         torch.tensor(tokenizer.convert_tokens_to_ids([tgt_token[m]])).to(device))
                Optimizer.zero_grad()
                if loss != 0:
                    total_loss.append((loss/len(MASK_idx)).item())
                    print(loss/len(MASK_idx), end='\r')
                    loss.backward()
                Optimizer.step()
            pbar2.update(1)
        pbar2.close()
        print("{}/{}\tLoss:{:.4f}".format(ep+1, EpochNum, np.mean(total_loss)))
        pbar1.update(1)
        
        ### Evaluation
        model.eval()
        
        correct, total = 0., 0
        correct_topk = np.zeros(10+1)
        perplexity_list = []
        CorrectFlag = False

        pbar = tqdm(total = len(valid_tgt))
        for i in range(len(valid_tgt)):
            MaskNum = np.sum(np.array(valid_src[i].split()) == "[MASK]")
            if MaskNum > MaskNumConstraint: # MASK Length Constraint
                pbar.update(1); continue
            elif MaskNum == 0:
                pbar.update(1); continue
            DataDict_valid = { "valid": Dataset.from_dict({ "text": [' '.join(valid_src[i].split()) ] }), }
            datasets_valid = DatasetDict(DataDict_valid)

            tokenized_datasets = datasets_valid.map(
                tokenize_function,
                batched=False,
                num_proc=1,
            )

            for cnt, x in enumerate(tokenized_datasets["valid"]):
                with torch.no_grad():
                    MASK_idx = [ii for ii, iids in enumerate(x["input_ids"])
                                if iids == tokenizer.convert_tokens_to_ids(["[MASK]"])[0]]
                    outputs = model(torch.tensor(x["input_ids"]).unsqueeze(0).to(device),)
                    
                    p = []
                    pred_token = valid_src[i].split()
                    tgt_token = valid_tgt[i].split()

                for m in MASK_idx:
                    predicted_index = torch.argmax(outputs["logits"][0][m]).item()
                    pred_t = tokenizer.convert_ids_to_tokens([predicted_index])[0]
#                     pred_token[m] = '[[' + pred_t + ']]' # When Visualization
                    pred_token[m] = pred_t
                    total += 1
                    if tgt_token[m] == pred_t:
                        CorrectFlag = True
                        correct += 1
                    ### Top-k
                    pred_topk = []; pred_ts = []
                    for n in range(1,10+1): # Starts from Top-1
                        pred_topk = torch.argsort(-outputs["logits"][0][m])[:n]
                        pred_ts = tokenizer.convert_ids_to_tokens([t for t in pred_topk])
                        if tgt_token[m] in pred_ts:
                            correct_topk[n] += 1

                    exp_t = outputs["logits"][0][m].cpu().numpy()
                    # https://huggingface.co/transformers/perplexity.html
                    prob = np.exp(exp_t[tokenizer.convert_tokens_to_ids(tgt_token[m])]/len(exp_t))
                    p.append(prob)
            if p: perplexity_list.append(np.sum(p))
            pbar.update(1)
        pbar.close()

        print(correct, '/', total, correct/total)
        for n in range(1,10+1):
            print("Top-"+str(n), "Accuracy", correct_topk[n]/total)
        print("Perplexity(MASK)", np.mean(perplexity_list))
        
        if correct/total > TopAcc:
            print("Model Saving in ./save/")
            model.module.save_pretrained('./save/')
            tokenizer.save_vocabulary("./save/")
            TopAcc = correct/total
    pbar1.close()
    ###

if Test:
    correct, total = 0., 0
    total_loss = []
    TopAcc = 0.
    correct_topk = np.zeros(10+1)
    perplexity_list = []
    CorrectFlag = False

    print("Model Loading from ./save/")
    model = AutoModelForMaskedLM.from_pretrained('./save/')
    model.resize_token_embeddings(len(tokenizer))
    model = nn.DataParallel(model).to(device)
    
    model.eval()
    pbar = tqdm(total = len(test_tgt))
    for i in range(len(test_tgt)):
        MaskNum = np.sum(np.array(test_src[i].split()) == "[MASK]")
        if MaskNum > MaskNumConstraint: # MASK Length Constraint
            pbar.update(1); continue
        elif MaskNum == 0:
            pbar.update(1); continue
        DataDict_test = { "test": Dataset.from_dict({ "text": [' '.join(test_src[i].split()) ] }), }
        datasets_test = DatasetDict(DataDict_test)

        tokenized_datasets = datasets_test.map(
            tokenize_function,
            batched=False,
            num_proc=1,
        )

        for cnt, x in enumerate(tokenized_datasets["test"]):
            with torch.no_grad():
                MASK_idx = [ii for ii, iids in enumerate(x["input_ids"])
                            if iids == tokenizer.convert_tokens_to_ids(["[MASK]"])[0]]
                outputs = model(torch.tensor(x["input_ids"]).unsqueeze(0).to(device),)
                p = []
                pred_token = test_src[i].split()
                tgt_token = test_tgt[i].split()

            for m in MASK_idx:
                predicted_index = torch.argmax(outputs["logits"][0][m]).item()
                pred_t = tokenizer.convert_ids_to_tokens([predicted_index])[0]
#                 pred_token[m] = '[[' + pred_t + ']]' # Visualization
                pred_token[m] = pred_t
                total += 1
                if tgt_token[m] == pred_t:
                    CorrectFlag = True
                    correct += 1
                ### Top-k
                pred_topk = []; pred_ts = []
                for n in range(1,10+1): # Starts from Top-1
                    pred_topk = torch.argsort(-outputs["logits"][0][m])[:n]
                    pred_ts = tokenizer.convert_ids_to_tokens([t for t in pred_topk])
                    if tgt_token[m] in pred_ts:
                        correct_topk[n] += 1

                exp_t = outputs["logits"][0][m].cpu().numpy()
                # https://huggingface.co/transformers/perplexity.html
                prob = np.exp(exp_t[tokenizer.convert_tokens_to_ids(tgt_token[m])]/len(exp_t))
                p.append(prob)
                
        if Verbose:
            if i <= 10:
                print("#Mask", MaskNum)
                ### Coloring
                for t1, t2 in zip(test_src[i].split(), pred_token):
                    if t1 == "[PAD]": break
                    else:
                        if t1 == "[MASK]":
                            print(colored(t2, 'red'), end=' ')
                        else:
                            print(t2, end=' ')
                print()
                #####
                for t1, t2 in zip(test_src[i].split(), test_tgt[i].split()):
                    if t1 == "[PAD]": break
                    else:
                        if t1 == "[MASK]":
                            print(colored(t2, 'blue'), end=' ')
                        else:
                            print(t2, end=' ')
                print()
                CorrectFlag = False

        if p: perplexity_list.append(np.sum(p))

        pbar.update(1)
    pbar.close()

    print(correct, '/', total, correct/total)
    for n in range(1,10+1):
        print("Top-"+str(n), "Accuracy", correct_topk[n]/total)
    print("Perplexity(MASK)", np.mean(perplexity_list))

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


/home/hyjo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:40: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


1/20	Loss:1.7341evice='cuda:0', grad_fn=<DivBackward0>))0>)



Top-1 Accuracy 140045.0 / 195005 0.7181610727930053
Top-1 Accuracy 0.7181610727930053
Top-2 Accuracy 0.7770159739493859
Top-3 Accuracy 0.8053485808056203
Top-4 Accuracy 0.8244968077741597
Top-5 Accuracy 0.8387477244173226
Top-6 Accuracy 0.8499423091715597
Top-7 Accuracy 0.8592497628265942
Top-8 Accuracy 0.8671572523781441
Top-9 Accuracy 0.8738647726981359
Top-10 Accuracy 0.8797415450885875
Perplexity(MASK) 4.770176012085848
Model Saving in ./save/Left3/4


2/20	Loss:1.2780evice='cuda:0', grad_fn=<DivBackward0>)d0>)


...


Top-1 Accuracy 143279.0 / 195005 0.7347452629419758
Top-1 Accuracy 0.7347452629419758
Top-2 Accuracy 0.7813902207635701
Top-3 Accuracy 0.8050306402399938
Top-4 Accuracy 0.8210917668777724
Top-5 Accuracy 0.8330350503833235
Top-6 Accuracy 0.8428553114022718
Top-7 Accuracy 0.850439732314556
Top-8 Accuracy 0.8569728981308171
Top-9 Accuracy 0.8627624932694034
Top-10 Accuracy 0.8678803107612625
Perplexity(MASK) 4.771026826873388


18/20	Loss:0.2762, device='cuda:0', grad_fn=<DivBackward0>)



Top-1 Accuracy 143031.0 / 195005 0.7334735006794697
Top-1 Accuracy 0.7334735006794697
Top-2 Accuracy 0.7802620445629599
Top-3 Accuracy 0.8041229712058665
Top-4 Accuracy 0.8198712853516577
Top-5 Accuracy 0.8315940616907259
Top-6 Accuracy 0.8406451116638035
Top-7 Accuracy 0.8484859362580447
Top-8 Accuracy 0.8549985897797492
Top-9 Accuracy 0.8607984410656137
Top-10 Accuracy 0.865588061844568
Perplexity(MASK) 4.77111567217422


19/20	Loss:0.2555, device='cuda:0', grad_fn=<DivBackward0>)



Top-1 Accuracy 142866.0 / 195005 0.7326273685290121
Top-1 Accuracy 0.7326273685290121
Top-2 Accuracy 0.77966718802082
Top-3 Accuracy 0.8031588933617086
Top-4 Accuracy 0.8190200251275608
Top-5 Accuracy 0.8307838260557422
Top-6 Accuracy 0.8400912797107767
Top-7 Accuracy 0.8477269813594523
Top-8 Accuracy 0.8539216943155303
Top-9 Accuracy 0.85969077715956
Top-10 Accuracy 0.8645983436322145
Perplexity(MASK) 4.771162248244989
